In [1]:
import pandas as pd
import numpy as np

from random import randint
import keras
import random
from sklearn.tree import export_graphviz
from sklearn import tree
from sklearn import metrics
from sklearn import tree
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn import linear_model
from sklearn import neighbors
from sklearn import neural_network
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support as score_all
from sklearn.metrics import confusion_matrix

from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
dataset = df = pd.read_excel ('SQI_Class.xlsx')

In [3]:
X = np.array(dataset.iloc[1:,1:7])
Y = np.array(dataset["Class"])

In [4]:
X_train_plus_valid, X_test, y_train_plus_valid, y_test  = train_test_split(X, Y, random_state=0, train_size = 0.9)

X_train, X_valid, y_train, y_valid  = train_test_split(X_train_plus_valid,y_train_plus_valid, random_state=0, train_size = 0.2/0.9)

In [5]:
train_mean=np.mean(X_train)
train_std=np.std(X_train)

In [6]:
X_train=(X_train-train_mean)/train_std
X_valid=(X_valid-train_mean)/train_std
X_test=(X_test-train_mean)/train_std

In [7]:
keras.backend.clear_session()

In [8]:
model = Sequential()
model.add(Dense(4,input_shape=(6,),activation='tanh'))
model.add(Dense(1, activation='linear'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 4)                 28        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 5         
Total params: 33
Trainable params: 33
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.save_weights('model.h5')

In [10]:
def train_model(X,Y,folds,model):
    accuracy_scores=[]
    precision_scores=[]
    sensitivity_scores=[]
    specificity_scores=[]
    f1_scores=[]
    for fold_n, (train_index, test_index) in enumerate(folds.split(X, Y)):
        X_train,X_test = X[train_index],X[test_index]
        y_train,y_test = Y[train_index],Y[test_index]
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        model.load_weights('model.h5')
        model.compile(loss='mean_squared_error',optimizer=keras.optimizers.adam(),metrics=['accuracy'])
        model.fit(X_train,y_train,batch_size=32,epochs=30,verbose=1)
        y_pred_test = model.predict(X_test).reshape(-1,)
        y_pred_test[y_pred_test<=0.5]=0
        y_pred_test[y_pred_test>0.5]=1
        cm1 = confusion_matrix(y_test, y_pred_test)
        precision,recall,fscore,support=score_all(y_test, y_pred_test,pos_label=0,average='binary')
        specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
        accuracy_scores.append(accuracy_score(y_test, y_pred_test))
        precision_scores.append(precision)
        sensitivity_scores.append(recall)
        specificity_scores.append(specificity)
        f1_scores.append(fscore)

    print('CV mean accuracy: {0:.4f}'.format(np.mean(accuracy_scores)))
    print('CV mean sensitivty: {0:.4f}'.format(np.mean(sensitivity_scores)))
    print('CV mean specificity: {0:.4f}'.format(np.mean(specificity_scores)))
    print('CV mean precision: {0:.4f}'.format(np.mean(precision_scores)))
    print('CV mean f1-score: {0:.4f}'.format(np.mean(f1_scores)))
    

In [11]:
folds = StratifiedKFold(10,shuffle=True,random_state=0)

In [12]:
#train_model(X,Y,folds,model)

In [13]:
rand_list = np.array(random.choices(range(0,X.shape[0]),k=3*X.shape[0]));
data1=X[rand_list[0:X.shape[0]],:];

In [14]:
data1

array([[17.27180714, 15.37614589, -3.36743432, -3.31925585, -3.02400096,
         1.        ],
       [22.92976648, 24.46654968,  3.71247052,  4.09732178,  4.24390016,
         1.        ],
       [ 7.07928285,  9.80695818,  0.68020427,  0.56104542, -1.78000931,
         0.        ],
       ...,
       [30.57002083, 34.70311176,  4.71931024,  4.94007729,  5.26460709,
         1.        ],
       [ 5.6544649 ,  5.78260824, -0.71041239, -0.88507892, -0.86657846,
         0.        ],
       [18.44298203, 19.70131857,  3.07324947,  3.5099471 ,  3.50756573,
         1.        ]])

In [15]:
def ensembling (X,Y,model1,model2,model3):
    rand_list = np.array(random.choices(range(0,X.shape[0]),k=3*X.shape[0]));
    data1=X[rand_list[0:X.shape[0]],:];
    classes1=Y[rand_list[0:X.shape[0]]];
    model1.load_weights('model.h5')
    model1.compile(loss='mean_squared_error',optimizer=keras.optimizers.adam(),metrics=['accuracy'])
    model1.fit(data1,classes1,batch_size=32,epochs=50,verbose=1)
    
    data1=X[rand_list[X.shape[0]+1:2*X.shape[0]],:];
    classes1=Y[rand_list[X.shape[0]+1:2*X.shape[0]]];
    model2.load_weights('model.h5')
    model2.compile(loss='mean_squared_error',optimizer=keras.optimizers.adam(),metrics=['accuracy'])
    model2.fit(data1,classes1,batch_size=32,epochs=50,verbose=1)
    
    data1=X[rand_list[2*X.shape[0]+1:3*X.shape[0]],:];
    classes1=Y[rand_list[2*X.shape[0]+1:3*X.shape[0]]];
    model3.load_weights('model.h5')
    model3.compile(loss='mean_squared_error',optimizer=keras.optimizers.adam(),metrics=['accuracy'])
    model3.fit(data1,classes1,batch_size=32,epochs=50,verbose=1)
    
    return model1,model2,model3
    

In [16]:
def ensembling_dec(model1,model2,model3,X):
    ped1=model1.predict(X)
    ped1[ped1<=0.5]=0
    ped1[ped1>0.5]=1
    ped2=model2.predict(X)
    ped2[ped2<=0.5]=0
    ped2[ped2>0.5]=1
    ped3=model3.predict(X)
    ped3[ped3<=0.5]=0
    ped3[ped3>0.5]=1
    tot=np.concatenate((ped1,ped2,ped3),axis=1)
    ped=np.median(tot, axis=1)
    return ped

In [17]:
def train_model_bag(X,Y,folds,model):
    cm1=np.zeros((2,2))
    cm2=np.zeros((2,2))
    accuracy_scores=[]
    #precision_scores=[]
    #sensitivity_scores=[]
    #specificity_scores=[]
    #f1_scores=[]
    for fold_n, (train_index, test_index) in enumerate(folds.split(X, Y)):
        X_train,X_test = X[train_index],X[test_index]
        y_train,y_test = Y[train_index],Y[test_index]
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        model.load_weights('model.h5')
        model.compile(loss='mean_squared_error',optimizer=keras.optimizers.adam(),metrics=['accuracy'])
        model1,model2,model3=ensembling(X_train,y_train,model,model,model)
        y_pred_test = ensembling_dec(model1,model2,model3,X_test)
        cm1 = cm1+confusion_matrix(y_test, y_pred_test)
       # precision,recall,fscore,support=score_all(y_test, y_pred_test,pos_label=0,average='binary')
       # specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
        print(accuracy_score(y_test,y_pred_test))
        accuracy_scores.append(accuracy_score(y_test, y_pred_test))
       # precision_scores.append(precision)
       # sensitivity_scores.append(recall)
       # specificity_scores.append(specificity)
       # f1_scores.append(fscore)
        y_pred_train = ensembling_dec(model1,model2,model3,X_train)
        cm2 = cm2+confusion_matrix(y_train, y_pred_train)
    
    accuracy=(cm1[1,1]+cm1[0,0])/(cm1[1,1]+cm1[0,0]+cm1[0,1]+cm1[1,0])
    sensitivity=cm1[0,0]/(cm1[0,0]+cm1[0,1])
    specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    precision=cm1[0,0]/(cm1[0,0]+cm1[1,0]);
    f1_score=(2*precision*sensitivity)/(precision+sensitivity)
    
    accuracy_t=(cm2[1,1]+cm2[0,0])/(cm2[1,1]+cm2[0,0]+cm2[0,1]+cm2[1,0])
    sensitivity_t=cm2[0,0]/(cm2[0,0]+cm2[0,1])
    specificity_t = cm2[1,1]/(cm2[1,0]+cm2[1,1])
    precision_t=cm2[0,0]/(cm2[0,0]+cm2[1,0]);
    f1_score_t=(2*precision_t*sensitivity_t)/(precision_t+sensitivity_t)
   # print('CV mean accuracy: {0:.4f}'.format(np.mean(accuracy_scores)))
   # print('CV mean sensitivty: {0:.4f}'.format(np.mean(sensitivity_scores)))
   # print('CV mean specificity: {0:.4f}'.format(np.mean(specificity_scores)))
   # print('CV mean precision: {0:.4f}'.format(np.mean(precision_scores)))
   # print('CV mean f1-score: {0:.4f}'.format(np.mean(f1_scores)))
    
    print('CV mean accuracy: {0:.4f}'.format(accuracy))
    print('CV mean sensitivty: {0:.4f}'.format(sensitivity))
    print('CV mean specificity: {0:.4f}'.format(specificity))
    print('CV mean precision: {0:.4f}'.format(precision))
    print('CV mean f1-score: {0:.4f}'.format(f1_score))
    
    print('CV mean accuracy t: {0:.4f}'.format(accuracy_t))
    print('CV mean sensitivty t: {0:.4f}'.format(sensitivity_t))
    print('CV mean specificity t: {0:.4f}'.format(specificity_t))
    print('CV mean precision t: {0:.4f}'.format(precision_t))
    print('CV mean f1-score t: {0:.4f}'.format(f1_score_t))
    
    
    return accuracy_scores
    

In [18]:
accuracy_scores_bag=train_model_bag(X,Y,folds,model)

Epoch 1/50
511/511 [==============================] - 0s 113us/step - loss: 0.4745 - accuracy: 0.3072
Epoch 2/50
511/511 [==============================] - 0s 20us/step - loss: 0.3759 - accuracy: 0.3640
Epoch 3/50
511/511 [==============================] - 0s 20us/step - loss: 0.3041 - accuracy: 0.4364
Epoch 4/50
511/511 [==============================] - 0s 20us/step - loss: 0.2515 - accuracy: 0.5675
Epoch 5/50
511/511 [==============================] - 0s 18us/step - loss: 0.2090 - accuracy: 0.6830
Epoch 6/50
511/511 [==============================] - 0s 20us/step - loss: 0.1743 - accuracy: 0.8023
Epoch 7/50
511/511 [==============================] - 0s 20us/step - loss: 0.1453 - accuracy: 0.8767
Epoch 8/50
511/511 [==============================] - 0s 18us/step - loss: 0.1212 - accuracy: 0.9061
Epoch 9/50
511/511 [==============================] - 0s 18us/step - loss: 0.1013 - accuracy: 0.9413
Epoch 10/50
511/511 [==============================] - 0s 18us/step - loss: 0.0850 - accur

In [19]:
accuracy_scores_bag

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9824561403508771,
 1.0,
 0.9824561403508771,
 1.0,
 0.9821428571428571]

In [20]:
keras.backend.clear_session()

In [21]:
model_lin = Sequential()
model_lin.add(Dense(4,input_shape=(6,),activation='tanh'))
model_lin.add(Dense(1, activation='linear'))
model_lin.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 4)                 28        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 5         
Total params: 33
Trainable params: 33
Non-trainable params: 0
_________________________________________________________________


In [22]:
model_lin.save_weights('model_lin.h5')

In [23]:
def grad_boosting (X,Y,modelg2,modelg3,modelg4):
    labels1=Y-0.5
    modelg2.load_weights('model_lin.h5')
    modelg2.compile(loss='mean_squared_error',optimizer=keras.optimizers.adam(),metrics=['accuracy'])
    modelg2.fit(X,labels1,batch_size=32,epochs=50,verbose=1)
    
    labels2=Y-0.5-modelg2.predict(X).reshape(Y.shape[0],)
    modelg3.load_weights('model_lin.h5')
    modelg3.compile(loss='mean_squared_error',optimizer=keras.optimizers.adam(),metrics=['accuracy'])
    modelg3.fit(X,labels2,batch_size=32,epochs=50,verbose=1)
    
    labels3=Y-0.5-modelg2.predict(X).reshape(Y.shape[0],)-modelg3.predict(X).reshape(Y.shape[0],)
    modelg4.load_weights('model_lin.h5')
    modelg4.compile(loss='mean_squared_error',optimizer=keras.optimizers.adam(),metrics=['accuracy'])
    modelg4.fit(X,labels3,batch_size=32,epochs=50,verbose=1)
    
    return modelg2,modelg3,modelg4

In [24]:
def train_model_boost(X,Y,folds,model_lin):
    accuracy_scores=[]
    #precision_scores=[]
    #sensitivity_scores=[]
    #specificity_scores=[]
    #f1_scores=[]
    cm1=np.zeros((2,2))
    cm2=np.zeros((2,2))
    for fold_n, (train_index, test_index) in enumerate(folds.split(X, Y)):
        X_train,X_test = X[train_index],X[test_index]
        y_train,y_test = Y[train_index],Y[test_index]
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        model_lin.load_weights('model_lin.h5')
        modelg1,modelg2,modelg3=grad_boosting(X_train,y_train,model_lin,model_lin,model_lin)
        y_pred_t = 0.5+modelg1.predict(X_train)+modelg2.predict(X_train)+modelg3.predict(X_train)
        y_pred_t[y_pred_t<=0.5]=0
        y_pred_t[y_pred_t>0.5]=1
        cm2= cm2+confusion_matrix(y_train,y_pred_t)
        
        y_pred_test = 0.5+modelg1.predict(X_test)+modelg2.predict(X_test)+modelg3.predict(X_test)
        y_pred_test[y_pred_test<=0.5]=0
        y_pred_test[y_pred_test>0.5]=1
        cm1 = cm1+confusion_matrix(y_test, y_pred_test)
        #precision,recall,fscore,support=score_all(y_test, y_pred_test,pos_label=0,average='binary')
        #specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
        print(accuracy_score(y_test, y_pred_test))
        accuracy_scores.append(accuracy_score(y_test, y_pred_test))
        #precision_scores.append(precision)
        #sensitivity_scores.append(recall)
        #specificity_scores.append(specificity)
        #f1_scores.append(fscore)
    
    accuracy=(cm1[1,1]+cm1[0,0])/(cm1[1,1]+cm1[0,0]+cm1[0,1]+cm1[1,0])
    sensitivity=cm1[0,0]/(cm1[0,0]+cm1[0,1])
    specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    precision=cm1[0,0]/(cm1[0,0]+cm1[1,0]);
    f1_score=(2*precision*sensitivity)/(precision+sensitivity)
    
    accuracy_t=(cm2[1,1]+cm2[0,0])/(cm2[1,1]+cm2[0,0]+cm2[0,1]+cm2[1,0])
    sensitivity_t=cm2[0,0]/(cm2[0,0]+cm2[0,1])
    specificity_t = cm2[1,1]/(cm2[1,0]+cm2[1,1])
    precision_t=cm2[0,0]/(cm2[0,0]+cm2[1,0]);
    f1_score_t=(2*precision_t*sensitivity_t)/(precision_t+sensitivity_t)
   # print('CV mean accuracy: {0:.4f}'.format(np.mean(accuracy_scores)))
   # print('CV mean sensitivty: {0:.4f}'.format(np.mean(sensitivity_scores)))
   # print('CV mean specificity: {0:.4f}'.format(np.mean(specificity_scores)))
   # print('CV mean precision: {0:.4f}'.format(np.mean(precision_scores)))
   # print('CV mean f1-score: {0:.4f}'.format(np.mean(f1_scores)))
    
    print('CV mean accuracy: {0:.4f}'.format(accuracy))
    print('CV mean sensitivty: {0:.4f}'.format(sensitivity))
    print('CV mean specificity: {0:.4f}'.format(specificity))
    print('CV mean precision: {0:.4f}'.format(precision))
    print('CV mean f1-score: {0:.4f}'.format(f1_score))
    
    print('CV mean accuracy t: {0:.4f}'.format(accuracy_t))
    print('CV mean sensitivty t: {0:.4f}'.format(sensitivity_t))
    print('CV mean specificity t: {0:.4f}'.format(specificity_t))
    print('CV mean precision t: {0:.4f}'.format(precision_t))
    print('CV mean f1-score t: {0:.4f}'.format(f1_score_t))
    

    
    
    return accuracy_scores
    

In [25]:
accuracy_scores_boost=train_model_boost(X,Y,folds,model_lin)

Epoch 1/50
511/511 [==============================] - 0s 119us/step - loss: 0.4125 - accuracy: 0.0000e+00
Epoch 2/50
511/511 [==============================] - 0s 20us/step - loss: 0.3335 - accuracy: 0.0000e+00
Epoch 3/50
511/511 [==============================] - 0s 21us/step - loss: 0.2813 - accuracy: 0.0000e+00
Epoch 4/50
511/511 [==============================] - 0s 19us/step - loss: 0.2463 - accuracy: 0.0000e+00
Epoch 5/50
511/511 [==============================] - 0s 19us/step - loss: 0.2188 - accuracy: 0.0000e+00
Epoch 6/50
511/511 [==============================] - 0s 19us/step - loss: 0.1977 - accuracy: 0.0000e+00
Epoch 7/50
511/511 [==============================] - 0s 20us/step - loss: 0.1777 - accuracy: 0.0000e+00
Epoch 8/50
511/511 [==============================] - 0s 18us/step - loss: 0.1613 - accuracy: 0.0000e+00
Epoch 9/50
511/511 [==============================] - 0s 18us/step - loss: 0.1457 - accuracy: 0.0000e+00
Epoch 10/50
511/511 [==============================] -

In [26]:
accuracy_scores_boost

[1.0,
 1.0,
 0.9824561403508771,
 0.9824561403508771,
 1.0,
 0.9824561403508771,
 1.0,
 0.9473684210526315,
 1.0,
 1.0]